# Thinking notebook

## Config

In [ ]:
from calib import calibrate_charuco
from utils import load_coefficients, save_coefficients
import cv2

import numpy as np
import cv2
import cv2.aruco as aruco
import matplotlib.pyplot as plt
import pathlib

%matplotlib inline
plt.rcParams["figure.figsize"] = (9,4)
plt.rcParams['figure.constrained_layout.use'] = True
plt.rcParams['image.cmap'] = 'gray'

## Methods (iterative...)

In [ ]:
def ray_plane_intersect(img_point, plane, cam_mtx):
	"""
	Calculate intersection between an image ray defined by `img_pont` and `cam_mtx`
	and a plane defined by the 4-vec plane
	"""
	assert len(plane) == 4
	assert len(img_point) == 2
	img_point = np.array([*img_point, 1])
	ray = np.linalg.inv(cam_mtx)@img_point
	p = plane[-1]
	n = plane[:3]
	return (-p/(n@ray)) * ray

In [ ]:
# RGB
def red_contrast(image):
	image = image.astype(np.float)
	return image[:,:, 0] - np.mean(image[:,:,1:], axis=-1)

def find_laser_points(image, tresh=150):
	contrast = red_contrast(image)

In [ ]:
def imshow(img, *args, **kwargs):
	ax = kwargs.get("ax")
	if ax:
		ax.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
	else:
		plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

In [ ]:
def mask_rgb(img, mask):
	ret = np.zeros_like(img)
	for i in range(ret.shape[-1]):
		ret[:,:,i] = img[:,:,i] * mask
	return ret

def get_charuco_rect(board):
	delim = np.max(board.chessboardCorners, axis=0) + board.getSquareLength() * np.array([1, 1, 0])
	corner_points = [
		[0, 0, 0],
		[delim[0], 0, 0],
		delim,
		[0, delim[1], 0],
	]
	return np.array(corner_points)

def get_charuco_mask(image, board, rvec, tvec, mtx, dist):
	if image.ndim==2:
		mask = np.zeros_like(image)
	elif image.ndim==3:
		mask = np.zeros(image.shape[:2])

	rect = get_charuco_rect(board)
	points, _ = cv2.projectPoints(rect, rvec, tvec, mtx, dist)
	proj_points_round = np.round(points, 0).astype(np.int)
	cv2.fillPoly(mask, [proj_points_round], True, 255 )
	return mask

def crop_charuco_board(image, board, rvec, tvec, mtx, dist):
	mask = get_charuco_mask(image, board, rvec, tvec, mtx, dist)
	if image.ndim == 3:
		return mask_rgb(image, mask)
	elif image.ndim==2:
		return mask*image

## Load params

In [ ]:
# def calibrate_charuco(dirpath, image_format, marker_length, square_length, prior=None, plot=False):
dirpath = r'Imagens\setupEnder_v0\\'
image_format = 'jpg'
# Dimensions in cm
marker_length = 2.1
square_length = 3.5
aruco_dict = aruco.Dictionary_get(aruco.DICT_6X6_1000)
board = aruco.CharucoBoard_create(5, 7, square_length, marker_length, aruco_dict)
arucoParams = aruco.DetectorParameters_create()
mtx, dist = load_coefficients('calibration_charuco.yml')


## Find charuco boards for each image

In [ ]:
counter, corners_list, id_list = [], [], []
imgs, imgs_rgb = [], []
img_dir = pathlib.Path(dirpath)
first = 0
# Find the ArUco markers inside each image
for img in img_dir.glob(f'*{image_format}'):
	# print(f'using image {img}')
	image = cv2.cvtColor(cv2.imread(str(img)), cv2.COLOR_BGR2RGB)
	img_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

	img_gray = cv2.undistort(img_gray, mtx, dist, None, mtx)
	image = cv2.undistort(image, mtx, dist, None, mtx)

	corners, ids, rejected = aruco.detectMarkers(
	img_gray, 
	aruco_dict, 
	parameters=arucoParams
	)
	resp, charuco_corners, charuco_ids = aruco.interpolateCornersCharuco(
	markerCorners=corners,
	markerIds=ids,
	image=img_gray,
	board=board
	)
	# If a Charuco board was found, let's collect image/corner points
	# Requiring at least 20 squares
	if resp > 10:
		corners_list.append(charuco_corners)
		id_list.append(charuco_ids)
		imgs.append(img_gray)
		imgs_rgb.append(image)

## Init Generator

In [ ]:
gen = (zip(corners_list, imgs, imgs_rgb, id_list))

In [ ]:
def next_fig():
	global points, img, img_rgb, ids, board, mtx, dist, rvec, tvec
	points, img, img_rgb, ids  = next(gen)
	ret, rvec, tvec = cv2.aruco.estimatePoseCharucoBoard(points, ids, board, mtx, dist, None, None)

## Calib laser

* rvec, tvec @ n_chessboard -> Plano em 3d do chessboard
* interseccao raio-plano com cam_mtx e n_chessboard -> Pontos 3d do laser
* (da pra fazer minimizacao tbm)
* Pontos 3d do laser em multiplos frames -> plano do laser

# Defining the chessboard plane

n_chessboard = np.array([0, 0, 1])

T_to_plane = np.eye(4)

M, jac = cv2.Rodrigues(rvec)

n3 = M@n_chessboard

p = -1 * n3@tvec

n = np.hstack((n3,p))

## Visualizar reprojection

In [ ]:
next_fig()
plt.figure()
proj_points, jacobian = cv2.projectPoints(board.chessboardCorners, rvec, tvec, mtx, dist)
plt.imshow(img_rgb)
plt.plot([p[0][0] for p in points], [p[0][1] for p in points], "r.", markersize=12)
plt.plot([p[0][0] for p in proj_points], [p[0][1] for p in proj_points], "g.", markersize=12)
plt.show(1)

# Visualizar find_laser

In [ ]:
next_fig()
plt.figure()
plt.imshow(red_contrast(img_rgb))

## Achar eixo principal laser

## Crop board

Metodo funciona bem, mas as vezes laser fora do board da ruim... Cropar o board!

In [ ]:
#def get_charuco_mask(image, board, rvec, tvec, mtx, dist):
next_fig()
fig, axs = plt.subplots(1,3)
mask = get_charuco_mask(img, board, rvec, tvec, mtx, dist)
axs[0].imshow(mask)

axs[1].imshow(crop_charuco_board(img_rgb, board, rvec, tvec, mtx, dist))
axs[2].imshow(img_rgb)

In [ ]:
def find_laser(img, std=5, crop=True):
	if crop:
		laser = red_contrast(crop_charuco_board(img,board,rvec,tvec,mtx,dist))
	else:
		laser = red_contrast(img)
	laser_norm = (laser - np.mean(laser)) / np.std(laser)
	return laser_norm > std

In [ ]:
#next_fig()
fig, axs = plt.subplots(1,2, sharex=True, sharey=True)
axs[0].imshow(img_rgb)
axs[1].imshow(find_laser(img_rgb, std=6))

## Eixo principal laser

In [ ]:
def find_laser_line(img, crop=True):
	"""
	Return a point indicating center of a line, and a vector with direction
	(coord. order is always x,y)
	"""
	laser_mask = find_laser(img, crop=crop)

	X = np.array(np.where(laser_mask)).T
	X = X[:,::-1]
	m = np.mean(X, axis=0)
	X_norm = X  - m
	u, s, v = np.linalg.svd(X_norm)
	V = v.T[:,0]

	return m, V

def line_line_intersection(v1, p1, v2, p2, lambdas = False):
	try:
		lamb1, lamb2 =  np.linalg.inv(np.array((v1.flatten(), -1*v2.flatten())).T) @ ((p2-p1).T)
	except np.linalg.LinAlgError:
		lamb1, lamb2 = 1e15, 1e15

	if lambdas:
		return lamb1, lamb2
	else:
		return [ (p1 + lamb1*v1), (p2+lamb2*v2) ]

In [ ]:
next_fig()
#plt.figure()
fig, ax = plt.subplots(1)
m, V = find_laser_line(img_rgb)
p0 = m - V*400
p1 = m + V*300
ax.imshow(img_rgb)
ax.plot([p0[0], p1[0]], [p0[1], p1[1]], "r")

In [ ]:
def find_laser_checkboard_intersects(img_rgb, board, rvec, tvec, mtx, dist):
	rect = get_charuco_rect(board)
	points, _ = cv2.projectPoints(rect, rvec, tvec, mtx, dist)

	m, v = find_laser_line(img_rgb)
	ps = []
	lambdas = []
	for p1, p2 in zip(points, np.roll(points, -1, axis=0)):
		m2 = (p1+p2)/2
		v2 = p2-p1
		ps.append(line_line_intersection(v,m, v2, m2, lambdas=False)[0])
		l1, l2 = line_line_intersection(v,m, v2, m2, lambdas=True)
		lambdas.append(l1)

	# The closest positive lambda 
	l0 = min([l for l in lambdas if l > 0])
	# The closest negative lambda
	l1 = max([l for l in lambdas if l < 0])

	x0 = (m + l0*v).flatten()
	x1 = (m + l1*v).flatten()
	return x0, x1

In [ ]:
next_fig()
plt.imshow(img_rgb)

# Plot corners
rect = get_charuco_rect(board)
points, _ = cv2.projectPoints(rect, rvec, tvec, mtx, dist)
plt.plot([p[0][0] for p in points],[p[0][1] for p in points], "r.")

# Plot laser line
x0, x1 = find_laser_checkboard_intersects(img_rgb, board, rvec, tvec, mtx, dist)
plt.plot((x0[0],x1[0]),(x0[1],x1[1]), "k")

# Plot sample points
vec_size = np.linalg.norm(x0-x1)
N = int(vec_size//20)
points = [x0 + (x1-x0)*i/N for i in range(1, N)]
plt.plot([x[0] for x in points],[x[1] for x in points], "r.")

In [ ]:
def bilinear_interpolate(im, x, y):
    x = np.asarray(x)
    y = np.asarray(y)

    x0 = np.floor(x).astype(int)
    x1 = x0 + 1
    y0 = np.floor(y).astype(int)
    y1 = y0 + 1

    x0 = np.clip(x0, 0, im.shape[1]-1);
    x1 = np.clip(x1, 0, im.shape[1]-1);
    y0 = np.clip(y0, 0, im.shape[0]-1);
    y1 = np.clip(y1, 0, im.shape[0]-1);

    Ia = im[ y0, x0 ]
    Ib = im[ y1, x0 ]
    Ic = im[ y0, x1 ]
    Id = im[ y1, x1 ]

    wa = (x1-x) * (y1-y)
    wb = (x1-x) * (y-y0)
    wc = (x-x0) * (y1-y)
    wd = (x-x0) * (y-y0)

    return wa*Ia + wb*Ib + wc*Ic + wd*Id

im = np.zeros((25,25))
im[20,14]=91
im[21,14]=162
im[20,15]=210
im[21,15]=95

bilinear_interpolate(im, 14.5, 20.2)

In [ ]:
# next_fig()
plt.imshow(img_rgb)

# Plot laser line
x0, x1 = find_laser_checkboard_intersects(img_rgb, board, rvec, tvec, mtx, dist)
plt.plot((x0[0],x1[0]),(x0[1],x1[1]), "k")

# Line, perp. line
m, v = find_laser_line(img_rgb)
vperp = np.array([-v[1], v[0]])

# Plot sample points
vec_size = np.linalg.norm(x0-x1)
N = int(vec_size//20)
points = [x0 + (x1-x0)*i/N for i in range(1, N)]
psup = [p - vperp*100 for p in points]
pinf = [p + vperp*100 for p in points]
for inf, sup in zip(pinf, psup):
	plt.plot([inf[0], sup[0]],[inf[1], sup[1]], "r")

In [ ]:
def find_laser_midpoint(img_rgb, point, vperp,):
	im = cv2.GaussianBlur(img_rgb, ksize=(5,5), sigmaX=1)
	samples = [point + vperp*l for l in np.linspace(-40,40,50)]

	intensities = bilinear_interpolate(red_contrast(im), [x[0] for x in samples], [x[1] for x in samples])

	intensities[(intensities - np.mean(intensities))/np.std(intensities) < 2] = 0 
	centroid = np.sum(intensities * np.arange(intensities.shape[0])) / np.sum(intensities)

	# max_int_point = samples[int(np.round(centroid))]
	max_intensity = np.argmax(intensities)
	max_int_point = samples[max_intensity]
	return max_int_point

In [ ]:
%matplotlib widget
p = points[8]

im = cv2.GaussianBlur(img_rgb, ksize=(5,5), sigmaX=1)

samples = [p + vperp*l for l in np.linspace(-40,40,50)]

intensities = bilinear_interpolate(red_contrast(im), [x[0] for x in samples], [x[1] for x in samples])
max_intensity = np.argmax(intensities)
max_int_point = samples[max_intensity]
print(max_int_point)

intensities[(intensities - np.mean(intensities))/np.std(intensities) < 2] = 0 
centroid = np.sum(intensities * np.arange(intensities.shape[0])) / np.sum(intensities)
plt.plot(intensities)
plt.axvline(max_intensity, color="g")
plt.axvline(centroid, color="b")

plt.figure()

plt.imshow(red_contrast(img_rgb))
plt.plot([x[0] for x in samples], [x[1] for x in samples], "r.")
plt.plot(max_int_point[0], max_int_point[1], "g.")

max_int_point = samples[int(np.round(centroid))]
plt.plot(max_int_point[0], max_int_point[1], "b.")

In [ ]:
# next_fig()
plt.imshow(img_rgb)

# Plot laser line
x0, x1 = find_laser_checkboard_intersects(img_rgb, board, rvec, tvec, mtx, dist)

# Line, perp. line
m, v = find_laser_line(img_rgb)
vperp = np.array([-v[1], v[0]])

# Find sampling points along laser
vec_size = np.linalg.norm(x0-x1)
N = int(vec_size//20)
points = [x0 + (x1-x0)*i/N for i in range(1, N)]

# Find laser image points precisely
precise_points = [find_laser_midpoint(img_rgb, p, vperp) for p in points]

# plot
plt.figure()
plt.imshow(img_rgb)
plt.plot([x[0] for x in points], [x[1] for x in points], "k.")
plt.plot([x[0] for x in precise_points], [x[1] for x in precise_points], "g.")

In [ ]:
def collect_laser_points(img_rgb, board, rvec, tvec, mtx, dist):
	# Find laser line limits in checkerboard
	x0, x1 = find_laser_checkboard_intersects(img_rgb, board, rvec, tvec, mtx, dist)

	# Find vector perp to laser line
	m, v = find_laser_line(img_rgb)
	vperp = np.array([-v[1], v[0]])

	# Find sampling points along laser
	vec_size = np.linalg.norm(x0-x1)
	N = int(vec_size//20)
	points = [x0 + (x1-x0)*i/N for i in range(1, N)]

	# Find laser image points precisely
	precise_points = [find_laser_midpoint(img_rgb, p, vperp) for p in points]

	return precise_points

## Triangular pontos com plano do chessboard

In [ ]:
def chess_plane(rvec, tvec):
	# Find p
	n_chessboard = np.array([0, 0, 1])
	M,_ = cv2.Rodrigues(rvec)
	n3 = M@n_chessboard
	p = -1 * n3@tvec
	n = np.hstack((n3,p))
	return n

In [ ]:
X = []
for points, img, img_rgb, ids in zip(corners_list, imgs, imgs_rgb, id_list):
	ret, rvec, tvec = cv2.aruco.estimatePoseCharucoBoard(points, ids, board, mtx, dist, None, None)

	points2d = collect_laser_points(img_rgb, board, rvec, tvec, mtx, dist)
	n = chess_plane(rvec, tvec)
	points3d = [ray_plane_intersect(p, n, mtx) for p in points2d]
	X.append(points3d)

In [ ]:
X = np.vstack(X)

In [ ]:
Xa = np.hstack((X, np.ones((X.shape[0], 1))))

In [ ]:
ax + by + cz + d = 0

In [ ]:
Xa

In [ ]:
u, s, v = np.linalg.svd(Xa)
print(s)
print(v)

In [ ]:
n = v.T[:,-1]
Xa @ n

In [ ]:
for points, img, img_rgb, ids in zip(corners_list, imgs, imgs_rgb, id_list):
	ret, rvec, tvec = cv2.aruco.estimatePoseCharucoBoard(points, ids, board, mtx, dist, None, None)

	points2d = collect_laser_points(img_rgb, board, rvec, tvec, mtx, dist)

	nc = chess_plane(rvec, tvec)

	points3d_chess = [ray_plane_intersect(p, nc, mtx) for p in points2d]
	points3d_triang = [ray_plane_intersect(p, n, mtx) for p in points2d]

	# print(np.array(points3d_chess) - np.array(points3d_triang))
	print(np.allclose(points3d_chess, points3d_triang, rtol=0.1))

In [ ]:
X.shape

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.scatter(Xa[:,0], Xa[:,1], Xa[:,2])

In [ ]:
def find_laser_borders(img_rgb):
	h, w,_ = img_rgb.shape
	points=np.array([
		[0,0],
		[w,0],
		[w,h],
		[0,h],
	])

	m, v = find_laser_line(img_rgb, crop=False)
	print(m,v)
	ps = []
	lambdas = []
	for p1, p2 in zip(points, np.roll(points, -1, axis=0)):
		m2 = (p1+p2)/2
		v2 = p2-p1
		ps.append(line_line_intersection(v,m, v2, m2, lambdas=False)[0])
		l1, l2 = line_line_intersection(v,m, v2, m2, lambdas=True)
		lambdas.append(l1)

	# The closest positive lambda 
	l0 = min([l for l in lambdas if l > 0])
	# The closest negative lambda
	l1 = max([l for l in lambdas if l < 0])

	x0 = (m + l0*v).flatten()
	x1 = (m + l1*v).flatten()
	return x0, x1

def find_laser_midpoint_horiz(img_rgb, x):
	im = cv2.GaussianBlur(img_rgb, ksize=(5,5), sigmaX=1)
	h, w = im.shape[:2]
	
	y = np.arange(0, h)
	x = np.ones_like(y) * x

	intensities = red_contrast(im)[y, x]
	intensities[(intensities - np.mean(intensities))/np.std(intensities) < 2] = 0 
	centroid = np.sum(intensities * np.arange(intensities.shape[0])) / np.sum(intensities)

	# max_int_point = samples[int(np.round(centroid))]
	max_intensity = np.argmax(intensities)
	max_int_point = (x[max_intensity], y[max_intensity])
	if (max_intensity > 0):
		return max_int_point
	
def find_points(img_rgb):
    # Find vector perp to laser line
	h, w = img_rgb.shape[:2]
	vperp = np.array([0, -1])

	# Find laser image points precisely
	precise_points = [find_laser_midpoint_horiz(img_rgb, x) for x in range(0, w, 10)]
	precise_points = [p for p in precise_points if p is not None]

	return precise_points

In [ ]:
# plt.imshow(image)
# plt.imshow(red_contrast(image))
plt.figure()
plt.imshow(find_laser(image, std=4, crop=False))
plt.figure()
plt.imshow(cv2.cvtColor(image, cv2.COLOR_RGB2GRAY) > 210)
plt.figure()
plt.hist(cv2.cvtColor(image, cv2.COLOR_RGB2GRAY).flatten(), bins=255)
plt.figure()
plt.imshow(image)

In [ ]:
%matplotlib qt5
# def calibrate_charuco(dirpath, image_format, marker_length, square_length, prior=None, plot=False):
dirpath = r'C:\Users\Pedro\Desktop\TCC\Code\Imagens\testEnder_v0\pics'
image_format = 'jpg'

p3d=[]
for img in pathlib.Path(dirpath).glob(f"*.{image_format}"):
	image = cv2.cvtColor(cv2.imread(str(img)), cv2.COLOR_BGR2RGB)
	img_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

	img_gray = cv2.undistort(img_gray, mtx, dist, None, mtx)
	image = cv2.undistort(image, mtx, dist, None, mtx)


	print("hi")
	try:
		laser_mask = find_laser(image, crop=False) | (img_gray > 210)
		points = find_points(image * laser_mask[:,:,np.newaxis])
		points_3d = [ray_plane_intersect(p, n, mtx) for p in points]
		if len(points_3d) ==0:
			plt.imshow(image)
			print(points)
			plt.plot([p[0] for p in points], [p[1] for p in points])
			plt.show()
			break
		p3d.append(points_3d)
		print(f"done with {str(img)}")
	except Exception as e:
		print(e)
		continue

In [ ]:
p3d
X=[]
for i, v in enumerate(p3d):
    print(len(v))
    if v:
        X.append(np.vstack(v)+ [0, i, 0])
X = np.vstack(X)

In [ ]:
X

In [ ]:
p3d = np.array(p3d)
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.scatter(X[:,0], X[:,1], X[:,2])

In [ ]:
gen = (zip(corners_list, imgs, imgs_rgb, id_list))